In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [89]:
xg_train = pd.read_csv('sample/dtrain_predprob169.csv')
xg_test = pd.read_csv('sample/dtest_predprob169.csv')
xg_val = pd.read_csv('sample/dval_predprob169.csv')
index = pd.read_csv('index.csv')
row_train = pd.read_csv('train.csv', sep=',')
row_val = pd.read_csv('validation.csv', sep=',')
row_test = pd.read_csv('test.csv', sep=',')

In [90]:
print(xg_train.shape)
print(row_train.shape)

(91443, 1)
(2430981, 25)


In [20]:
x_tr4 = pd.DataFrame(xg_train)
print(x_tr4.shape)
x_test4 = pd.DataFrame(xg_test)
x_val4 = pd.DataFrame(xg_val)
clicks = len(row_train[row_train.click==1])
impression = len(row_train[row_train.bidprice>row_train.payprice])
aCTR=clicks/impression
x_tr4 = x_tr4/aCTR
x_test4 = x_test4/aCTR
x_val4 = x_val4/aCTR
index = index['0']
row_train.payprice[row_train['click']==0] = 0
y_tr4 = row_train.payprice[index]
y_val4 = row_val.payprice

(91443, 1)


/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [21]:
model = Sequential()
# Adding the input layer and the first hidden layer
model.add(Dense(output_dim = 100, init = 'normal', activation ='relu',input_dim = 1))
# Adding the second hidden layer
model.add(Dense(output_dim = 100, activation ='relu',init = 'normal'))
# Adding the output layer
model.add(Dense(output_dim = 1,init = 'normal'))
model.compile(optimizer = 'adam', loss = 'mean_absolute_error', metrics = ['accuracy'])
history = model.fit(x_tr4, y_tr4, batch_size = 1, nb_epoch = 1)

/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=1, units=100, kernel_initializer="normal")`
  app.launch_new_instance()
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="normal")`
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=1, kernel_initializer="normal")`
/opt/anaconda/envs/Python3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/1
91443/91443 [==============================] - 4426s 48ms/step - loss: 1.8718 - acc: 0.9711


In [23]:
train_score = model.evaluate(x_tr4, y_tr4, verbose=0)
valid_score = model.evaluate(x_val4, y_val4, verbose=0)

print('Train MAE: ', round(train_score[1], 4), ', Train Loss: ', round(train_score[0], 4)) 
print('Val MAE: ', round(valid_score[1], 4), ', Val Loss: ', round(valid_score[0], 4))

Train MAE:  0.979 , Train Loss:  1.9075
Val MAE:  0.0003 , Val Loss:  78.2093


In [82]:
y_pred_test2 = model.predict(x_test4)
y_pred_tr2 = model.predict(x_tr4)
y_pred_val2 = model.predict(x_val4)
display(y_pred_tr2)
display(x_tr4.head(10))
display(y_tr4.head(10))
y_pred_val2 = pd.DataFrame(y_pred_val2)
y_pred_tr2 = pd.DataFrame(y_pred_tr2)
y_pred_test2 = pd.DataFrame(y_pred_test2)
y_pred_test2.to_csv('ind_pred_bidprice.csv')
y_pred_val2.to_csv('ind_val_bidprice.csv')
y_pred_tr2.to_csv('ind_train_bidprice.csv')

array([[-0.00015761],
       [-0.00015761],
       [-0.00015761],
       ...,
       [-0.00015761],
       [-0.00015761],
       [-0.00983362]], dtype=float32)

,pCTR
0,5.851756
1,13.762963
2,9.773087
3,347.844727
4,9.564919
5,5.394247
6,8.520240
7,5.972116
8,6.975187
9,16.503791


224827     0
2238536    0
1395970    0
1330632    0
397740     0
45265      0
1065063    0
2307453    0
389379     0
1011890    0
Name: payprice, dtype: int64

In [87]:
y_pred_val4 = pd.DataFrame(y_pred_val2)
y_pred_val4[y_pred_val3>0] = y_pred_val4[y_pred_val3>0]+300
print(y_pred_val4[1701:1705])

def evalu(pred, data):
    budget=6250000
    impression=0
    click=0
    
    for row in range(0,data.shape[0]):
        if budget<data.payprice[row]:
            continue
        if pred.iloc[row,0]>data.payprice[row]:
            budget=budget-data.payprice[row]
            impression=impression+1
            if data.click[row]==1:
                click=click+1
    print('budget',budget)
    print('impression',impression )
    return click

a = evalu(y_pred_val4, row_val)
print('clicks',a)

                0
1701    -0.000158
1702  1527.749756
1703    -0.000158
1704    -0.000158
budget 6195099
impression 585
clicks 72


In [56]:
def bad_ctr(x):
    new = pd.crosstab(index=x, columns=row_train['click'])
    new['Clicks']=new[1]
    new['Imps'] = pd.crosstab(index=x, columns='counts')

    
#CTR: Click-Through Rate = #clicks / #imps advertise   (per Advertiser)
    new['CTR']=(new[1]/new['Imps'])*100
    new['best']=new.CTR>=0.015
#hours.CTR.value_counts().plot(kind='bar')
    #display(new)
    #indexes = [index for index in (new) if new['best'][index] == False]
    indexes= new[new['best']==False]
    return indexes.index.values

In [57]:
bad_cities = bad_ctr(row_train['city'])
#bad_days = bad_ctr(row_train['weekday'])
bad_hours = bad_ctr(row_train['hour'])
bad_region = bad_ctr(row_train['region'])
bad_slot_width = bad_ctr(row_train['slotwidth'])
bad_slot_height = bad_ctr(row_train['slotheight'])
print(bad_slot_height)

[100 150 230]


In [80]:
index =[]
y_pred_val3 = y_pred_val2
y_pred_val3[y_pred_val3>0] = y_pred_val3[y_pred_val3>0]+70
def evaluate(bidprice):
    budget=6250000
    impression=0
    click=0
    for row in range(0,row_val.shape[0]):
        if budget<row_val.payprice[row]:
            continue
        if row_val.city[row] not in bad_cities:
            if row_val.hour[row] not in bad_hours:
                if row_val.region[row] not in bad_region:
                    if row_val.slotwidth[row] not in bad_slot_width:
                        if row_val.slotheight[row] not in bad_slot_height:
                            if bidprice.iloc[row,0]>row_val.payprice[row]:
                                budget=budget-row_val.payprice[row]
                                impression=impression+1
                                if row_val.click[row]==1:
                                    click=click+1
                                    index.append(row)
    print ('budget',budget)                                
    return click

new_clicks = evaluate(y_pred_val2)
print('clicks',new_clicks)
print(index)

KeyboardInterrupt: 

In [70]:
print(y_pred_val2)
print(row_val[row_val['click']==1])

               0
0      -0.000158
1      -0.000158
2      -0.000158
3      -0.000158
4      -0.000158
5      -0.000158
6      -0.000158
7      -0.000158
8      -0.000158
9      -0.000158
10     -0.000158
11     -0.000158
12     -0.000158
13     -0.000158
14     -0.000158
15     -0.000158
16     -0.000158
17     -0.000158
18     -0.000158
19     -0.000158
20     -0.000158
21     -0.000158
22     -0.000158
23     -0.000158
24     -0.000158
25     -0.000158
26     -0.000158
27     -0.000158
28     -0.000158
29     -0.000158
...          ...
303895 -0.000495
303896 -0.000158
303897 -0.000158
303898 -0.000158
303899 -0.000158
303900 -0.000158
303901 -0.000158
303902 -0.000158
303903 -0.000158
303904 -0.000158
303905 -0.000158
303906 -0.000158
303907 -0.000158
303908 -0.000158
303909 -0.000158
303910 -0.000158
303911 -0.000158
303912 -0.000158
303913 -0.000158
303914 -0.000158
303915 -0.000158
303916 -0.000158
303917 -0.000158
303918 -0.000158
303919 -0.000158
303920 -0.000158
303921 -0.0001

In [78]:
print(y_pred_val2[1701:1705])

              0
1701  -0.000158
1702  27.749792
1703  -0.000158
1704  -0.000158
